In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_80893/3522731578.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
import mysql.connector
from collections import defaultdict

##############################################################################
# 1) Database Query for Quarterfinal/Conference/Championship Payouts
##############################################################################
# (This assumes your DB has "NBA" bets with EventType 
# in {'Quarterfinals', 'Conference Winner', 'Championship'}. 
# Adjust as needed for your environment.)

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)
cursor = conn.cursor()

query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NBA'
        AND legs.EventType IN ('Quarterfinals', 'Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary: keys are team names (lower-case)
payouts = defaultdict(lambda: {
    'quarterfinals_payout': 0.0,
    'semifinal_payout': 0.0,   # for "Conference Winner"
    'championship_payout': 0.0
})

for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    etype = event_type.strip().lower()
    if etype == "quarterfinals":
        payouts[normalized_name]['quarterfinals_payout'] = float(total_payout)
    elif etype == "conference winner":
        payouts[normalized_name]['semifinal_payout'] = float(total_payout)
    elif etype == "championship":
        payouts[normalized_name]['championship_payout'] = float(total_payout)

cursor.close()
conn.close()

##############################################################################
# 2) Teams & Hypothetical Probabilities
##############################################################################
# 8 NBA teams, each with "current_round_prob" for QF 
# and "hypothetical_probabilities" for potential future matchups.
# We have 4 teams in East, 4 in West.

teams = {
    # Eastern
    "milwaukee bucks":      {"current_round_prob": 0.7},
    "chicago bulls":        {"current_round_prob": 0.3},
    "boston celtics":       {"current_round_prob": 0.45},
    "miami heat":           {"current_round_prob": 0.52},
    # Western
    "denver nuggets":       {"current_round_prob": 0.6},
    "golden state warriors":{"current_round_prob": 0.4},
    "los angeles lakers":   {"current_round_prob": 0.55},
    "phoenix suns":         {"current_round_prob": 0.48}
}

# Pairwise matchup probabilities for possible semifinal / championship matchups.
# The lines for QF pairs also appear but won't be used for the QF stage 
# because "current_round_prob" handles that directly.
hypothetical_probabilities = {
    "milwaukee bucks": {
        "chicago bulls":         0.5,
        "boston celtics":        0.5,
        "miami heat":            0.55,
        "denver nuggets":        0.55,
        "golden state warriors": 0.65,
        "los angeles lakers":    0.5,
        "phoenix suns":          0.52
    },
    "chicago bulls": {
        "milwaukee bucks":       0.5,
        "boston celtics":        0.55,
        "miami heat":            0.52,
        "denver nuggets":        0.6,
        "golden state warriors": 0.45,
        "los angeles lakers":    0.48,
        "phoenix suns":          0.48
    },
    "boston celtics": {
        "milwaukee bucks":       0.5,
        "chicago bulls":         0.45,
        "miami heat":            0.55,
        "denver nuggets":        0.58,
        "golden state warriors": 0.52,
        "los angeles lakers":    0.5,
        "phoenix suns":          0.5
    },
    "miami heat": {
        "milwaukee bucks":       0.45,
        "chicago bulls":         0.48,
        "boston celtics":        0.45,
        "denver nuggets":        0.5,
        "golden state warriors": 0.5,
        "los angeles lakers":    0.4,
        "phoenix suns":          0.45
    },
    "denver nuggets": {
        "golden state warriors": 0.5,
        "los angeles lakers":    0.45,
        "phoenix suns":          0.53,
        "milwaukee bucks":       0.45,
        "chicago bulls":         0.4,
        "boston celtics":        0.58,
        "miami heat":            0.5
    },
    "golden state warriors": {
        "denver nuggets":        0.5,
        "los angeles lakers":    0.55,
        "phoenix suns":          0.55,
        "milwaukee bucks":       0.35,
        "chicago bulls":         0.55,
        "boston celtics":        0.48,
        "miami heat":            0.5
    },
    "los angeles lakers": {
        "denver nuggets":        0.55,
        "golden state warriors": 0.45,
        "phoenix suns":          0.49,
        "milwaukee bucks":       0.5,
        "chicago bulls":         0.52,
        "boston celtics":        0.5,
        "miami heat":            0.6
    },
    "phoenix suns": {
        "denver nuggets":        0.47,
        "golden state warriors": 0.45,
        "los angeles lakers":    0.51,
        "milwaukee bucks":       0.48,
        "chicago bulls":         0.52,
        "boston celtics":        0.5,
        "miami heat":            0.55
    }
}

##############################################################################
# 3) Define Conferences & QF Pairs
##############################################################################
conference_east = [
    "milwaukee bucks",
    "chicago bulls",
    "boston celtics",
    "miami heat"
]

conference_west = [
    "denver nuggets",
    "golden state warriors",
    "los angeles lakers",
    "phoenix suns"
]

def create_qf_pairs(conf_teams):
    """Given exactly 4 teams in one conference, return two QF pairs in bracket order."""
    return [
        (conf_teams[0], conf_teams[1]),
        (conf_teams[2], conf_teams[3])
    ]

east_pairs = create_qf_pairs(conference_east)
west_pairs = create_qf_pairs(conference_west)

##############################################################################
# 4) Bracket EV Functions
##############################################################################

def quarterfinals_ev(winner, same_conference):
    pairs = create_qf_pairs(same_conference)
    if winner in pairs[0]:
        other_pair = pairs[1]
    else:
        other_pair = pairs[0]
    qf_ev = teams[winner]["current_round_prob"] * payouts[winner]["quarterfinals_payout"]
    for x in other_pair:
        qf_ev += teams[x]["current_round_prob"] * payouts[x]["quarterfinals_payout"]
    return qf_ev

def this_conference_ev(winner, same_conference):
    pairs = create_qf_pairs(same_conference)
    if winner in pairs[0]:
        other_pair = pairs[1]
    else:
        other_pair = pairs[0]

    conf_ev = 0.0
    for x in other_pair:
        p_x_qf      = teams[x]["current_round_prob"]
        p_t_beats_x = hypothetical_probabilities[winner][x]
        p_x_beats_t = hypothetical_probabilities[x][winner]
        conf_ev += (
            p_x_qf * (
                p_t_beats_x * payouts[winner]["semifinal_payout"]
              + p_x_beats_t * payouts[x]["semifinal_payout"]
            )
        )
    return conf_ev

def other_conference_ev(other_conference):
    pairs = create_qf_pairs(other_conference)

    def p_conf_champ(team_name):
        # Probability team_name wins its QF times sum(prob opp wins QF * team beats opp)
        if team_name in pairs[0]:
            z_other = pairs[1]
        else:
            z_other = pairs[0]
        p_z_qf = teams[team_name]["current_round_prob"]
        sum_prob = 0.0
        for opp in z_other:
            p_opp_qf = teams[opp]["current_round_prob"]
            p_z_beats_opp = hypothetical_probabilities[team_name][opp]
            sum_prob += p_opp_qf * p_z_beats_opp
        return p_z_qf * sum_prob

    oc_ev = 0.0
    for z in other_conference:
        oc_ev += p_conf_champ(z) * payouts[z]["semifinal_payout"]
    return oc_ev

def championship_ev(winner, same_conference, other_conference):
    pairs = create_qf_pairs(same_conference)
    if winner in pairs[0]:
        other_pair = pairs[1]
    else:
        other_pair = pairs[0]

    # Probability "winner" emerges from T's side
    p_T_conf_champ = 0.0
    for x in other_pair:
        p_x_qf      = teams[x]["current_round_prob"]
        p_T_beats_x = hypothetical_probabilities[winner][x]
        p_T_conf_champ += p_x_qf * p_T_beats_x

    def p_conf_champ_x(x):
        # Probability x wins QF * x beats T
        return teams[x]["current_round_prob"] * hypothetical_probabilities[x][winner]

    p_conf_winner_T_side = {winner: p_T_conf_champ}
    for x in other_pair:
        p_conf_winner_T_side[x] = p_conf_champ_x(x)

    # Probability each team in other conference emerges
    oc_pairs = create_qf_pairs(other_conference)
    def p_conf_champ_z(z):
        if z in oc_pairs[0]:
            z_other = oc_pairs[1]
        else:
            z_other = oc_pairs[0]
        p_z_qf = teams[z]["current_round_prob"]
        sum_z_wins_sf = 0.0
        for opp in z_other:
            p_opp_qf      = teams[opp]["current_round_prob"]
            p_z_beats_opp = hypothetical_probabilities[z][opp]
            sum_z_wins_sf += p_opp_qf * p_z_beats_opp
        return p_z_qf * sum_z_wins_sf

    p_conf_winner_other_side = {
        z: p_conf_champ_z(z) for z in other_conference
    }

    champ_ev = 0.0
    for y, p_y_conf_champ in p_conf_winner_T_side.items():
        for z, p_z_conf_champ in p_conf_winner_other_side.items():
            p_final = p_y_conf_champ * p_z_conf_champ
            p_y_champ = hypothetical_probabilities[y][z]
            p_z_champ = hypothetical_probabilities[z][y]
            champ_ev += p_final * (
                p_y_champ * payouts[y]["championship_payout"]
              + p_z_champ * payouts[z]["championship_payout"]
            )
    return champ_ev

def calculate_portfolio_ev(winner, same_conference, other_conference):
    qf_ev = quarterfinals_ev(winner, same_conference)
    conf_ev = this_conference_ev(winner, same_conference)
    oc_ev = other_conference_ev(other_conference)
    c_ev  = championship_ev(winner, same_conference, other_conference)
    return qf_ev + conf_ev + oc_ev + c_ev

##############################################################################
# 5) Unconditional Tournament Portfolio EV
##############################################################################
def calculate_unconditional_tournament_ev():
    """
    Sums each team's (quarterfinal EV + conference EV + championship EV) 
    in an unconditional scenario.
    """
    total_ev = 0.0
    all_teams_list = list(teams.keys())
    for t in all_teams_list:
        p_qf = teams[t]["current_round_prob"]

        # Identify t's bracket
        if t in conference_east:
            conf = conference_east
            other_conf = conference_west
        else:
            conf = conference_west
            other_conf = conference_east

        # Identify which QF pair t is in
        qf_pairs_local = create_qf_pairs(conf)
        if t in qf_pairs_local[0]:
            opp_pair = qf_pairs_local[1]
        else:
            opp_pair = qf_pairs_local[0]

        # Probability t wins the conference
        sum_t_beats = 0.0
        for opp in opp_pair:
            sum_t_beats += teams[opp]["current_round_prob"] * hypothetical_probabilities[t][opp]
        p_conf = p_qf * sum_t_beats

        # Probability t wins the championship
        sum_t_final = 0.0
        # For each possible team z in the other conference
        qf_pairs_other = create_qf_pairs(other_conf)
        for z in other_conf:
            if z in qf_pairs_other[0]:
                z_opp_pair = qf_pairs_other[1]
            else:
                z_opp_pair = qf_pairs_other[0]

            sum_z_beats = 0.0
            for o in z_opp_pair:
                sum_z_beats += teams[o]["current_round_prob"] * hypothetical_probabilities[z][o]
            p_conf_z = teams[z]["current_round_prob"] * sum_z_beats

            sum_t_final += p_conf_z * hypothetical_probabilities[t][z]

        p_champ = p_conf * sum_t_final

        # sum up the quarterfinal, conference, and championship EV for t
        total_ev += (
            p_qf  * payouts[t]["quarterfinals_payout"]
          + p_conf* payouts[t]["semifinal_payout"]
          + p_champ * payouts[t]["championship_payout"]
        )
    return total_ev

##############################################################################
# 6) Conditional EV: forcing a QF winner
##############################################################################
def calculate_conditional_portfolio_ev(winner, same_conference, other_conference, conditioned_team):
    """
    For the QF in which 'conditioned_team' plays, we set it to 1.0 (winner) 
    and its QF opponent(s) to 0.0, then recalc. Then revert.
    """
    # Identify the bracket in which 'conditioned_team' belongs
    if conditioned_team in same_conference:
        conf_of_cond = same_conference
    else:
        conf_of_cond = other_conference

    qf_pairs = create_qf_pairs(conf_of_cond)
    found_pair = None
    for pair in qf_pairs:
        if conditioned_team in pair:
            found_pair = pair
            break

    # If not found, just do normal
    if not found_pair:
        return calculate_portfolio_ev(winner, same_conference, other_conference)

    # Save old probabilities
    old_probs = {tm: teams[tm]["current_round_prob"] for tm in found_pair}

    # Force 'conditioned_team' to 1.0, the other to 0.0
    for tm in found_pair:
        if tm == conditioned_team:
            teams[tm]["current_round_prob"] = 1.0
        else:
            teams[tm]["current_round_prob"] = 0.0

    # compute EV
    ev = calculate_portfolio_ev(winner, same_conference, other_conference)

    # revert
    for tm in found_pair:
        teams[tm]["current_round_prob"] = old_probs[tm]

    return ev

##############################################################################
# 7) Forced QF Outcomes (16 scenarios)
##############################################################################
E1A, E1B = east_pairs[0]
E2A, E2B = east_pairs[1]
W1A, W1B = west_pairs[0]
W2A, W2B = west_pairs[1]

all_scenarios = {}

def force_qf_outcome(winner, loser):
    old_w = teams[winner]["current_round_prob"]
    old_l = teams[loser]["current_round_prob"]
    teams[winner]["current_round_prob"] = 1.0
    teams[loser]["current_round_prob"]  = 0.0
    return (old_w, old_l)

def revert_qf_outcome(winner, loser, ow, ol):
    teams[winner]["current_round_prob"] = ow
    teams[loser]["current_round_prob"]  = ol

for e1_winner in (E1A, E1B):
    e1_loser = E1B if e1_winner == E1A else E1A
    ow_e1w, ow_e1l = force_qf_outcome(e1_winner, e1_loser)

    for e2_winner in (E2A, E2B):
        e2_loser = E2B if e2_winner == E2A else E2A
        ow_e2w, ow_e2l = force_qf_outcome(e2_winner, e2_loser)

        for w1_winner in (W1A, W1B):
            w1_loser = W1B if w1_winner == W1A else W1A
            ow_w1w, ow_w1l = force_qf_outcome(w1_winner, w1_loser)

            for w2_winner in (W2A, W2B):
                w2_loser = W2B if w2_winner == W2A else W2A
                ow_w2w, ow_w2l = force_qf_outcome(w2_winner, w2_loser)

                scenario_key = (
                    f"E1={e1_winner.title()}, E2={e2_winner.title()}, "
                    f"W1={w1_winner.title()}, W2={w2_winner.title()}"
                )

                bracket_ev = 0.0
                for tm in teams.keys():
                    p_qf = teams[tm]["current_round_prob"]

                    if tm in conference_east:
                        local_pairs = create_qf_pairs(conference_east)
                        other_conf  = conference_west
                    else:
                        local_pairs = create_qf_pairs(conference_west)
                        other_conf  = conference_east

                    if tm in local_pairs[0]:
                        opp_pair = local_pairs[1]
                    else:
                        opp_pair = local_pairs[0]

                    # p_conf for tm
                    sum_tm = 0.0
                    for opp in opp_pair:
                        sum_tm += teams[opp]["current_round_prob"] * hypothetical_probabilities[tm][opp]
                    p_conf = p_qf * sum_tm

                    # p_champ for tm
                    qf_pairs_other = create_qf_pairs(other_conf)
                    sum_final = 0.0
                    for z in other_conf:
                        if z in qf_pairs_other[0]:
                            z_opp_pair = qf_pairs_other[1]
                        else:
                            z_opp_pair = qf_pairs_other[0]

                        sum_z = 0.0
                        for o in z_opp_pair:
                            sum_z += teams[o]["current_round_prob"] * hypothetical_probabilities[z][o]
                        p_conf_z = teams[z]["current_round_prob"] * sum_z
                        sum_final += p_conf_z * hypothetical_probabilities[tm][z]

                    p_champ = p_conf * sum_final

                    bracket_ev += (
                        p_qf  * payouts[tm]["quarterfinals_payout"]
                      + p_conf* payouts[tm]["semifinal_payout"]
                      + p_champ* payouts[tm]["championship_payout"]
                    )

                all_scenarios[scenario_key] = bracket_ev

                revert_qf_outcome(w2_winner, w2_loser, ow_w2w, ow_w2l)
            revert_qf_outcome(w1_winner, w1_loser, ow_w1w, ow_w1l)
        revert_qf_outcome(e2_winner, e2_loser, ow_e2w, ow_e2l)
    revert_qf_outcome(e1_winner, e1_loser, ow_e1w, ow_e1l)

##############################################################################
# 8) Scenario-Based EV for Each "Winner"
##############################################################################
ev_results = {}
for t in teams.keys():
    if t in conference_east:
        ev_results[t] = calculate_portfolio_ev(t, conference_east, conference_west)
    else:
        ev_results[t] = calculate_portfolio_ev(t, conference_west, conference_east)

##############################################################################
# 9) Quarterfinal Matchups: EV Deltas
##############################################################################
def qf_delta(pair):
    t1, t2 = pair
    return abs(ev_results[t1] - ev_results[t2])

east_delta1 = qf_delta(east_pairs[0])
east_delta2 = qf_delta(east_pairs[1])
west_delta1 = qf_delta(west_pairs[0])
west_delta2 = qf_delta(west_pairs[1])

##############################################################################
# 10) Unconditional EV
##############################################################################
uncond_ev = calculate_unconditional_tournament_ev()

##############################################################################
# 11) Conditional EV (If "Team X" Already Won Another QF)
##############################################################################
from collections import defaultdict

conditional_ev_results = defaultdict(dict)
for winner in teams.keys():
    if winner in conference_east:
        sc = conference_east
        oc = conference_west
    else:
        sc = conference_west
        oc = conference_east

    # For each possible 'conditioned_team', force them to have won their QF
    for conditioned_team in teams.keys():
        conditional_ev_results[winner][conditioned_team] = (
            calculate_conditional_portfolio_ev(winner, sc, oc, conditioned_team)
        )

##############################################################################
# 12) Print Everything
##############################################################################
print("==== Scenario-Based Portfolio EVs (Team Wins Its QF) ====")
# Alphabetically
for team in sorted(ev_results.keys()):
    print(f"EV if {team.title()} is QF winner: ${ev_results[team]:,.2f}")

print("\n==== EV Deltas for Quarterfinal Matchups ====")
print(f"East QF: {east_pairs[0][0].title()} vs. {east_pairs[0][1].title()} => EV Delta = ${east_delta1:,.2f}")
print(f"East QF: {east_pairs[1][0].title()} vs. {east_pairs[1][1].title()} => EV Delta = ${east_delta2:,.2f}")
print(f"West QF: {west_pairs[0][0].title()} vs. {west_pairs[0][1].title()} => EV Delta = ${west_delta1:,.2f}")
print(f"West QF: {west_pairs[1][0].title()} vs. {west_pairs[1][1].title()} => EV Delta = ${west_delta2:,.2f}")

print("\n===== ALL 16 QF OUTCOMES => Portfolio EV =====\n")
for scenario, val in sorted(all_scenarios.items()):
    print(f"{scenario}:  EV=${val:,.2f}")

print("\n==== Unconditioned Tournament Portfolio EV ====")
print(f"Fully unconditional EV: ${uncond_ev:,.2f}")

==== Scenario-Based Portfolio EVs (Team Wins Its QF) ====
EV if Boston Celtics is QF winner: $133,095.54
EV if Chicago Bulls is QF winner: $73,209.74
EV if Denver Nuggets is QF winner: $120,482.42
EV if Golden State Warriors is QF winner: $198,246.27
EV if Los Angeles Lakers is QF winner: $148,911.23
EV if Miami Heat is QF winner: $174,098.52
EV if Milwaukee Bucks is QF winner: $185,178.63
EV if Phoenix Suns is QF winner: $146,354.95

==== EV Deltas for Quarterfinal Matchups ====
East QF: Milwaukee Bucks vs. Chicago Bulls => EV Delta = $111,968.89
East QF: Boston Celtics vs. Miami Heat => EV Delta = $41,002.97
West QF: Denver Nuggets vs. Golden State Warriors => EV Delta = $77,763.85
West QF: Los Angeles Lakers vs. Phoenix Suns => EV Delta = $2,556.28

===== ALL 16 QF OUTCOMES => Portfolio EV =====

E1=Chicago Bulls, E2=Boston Celtics, W1=Denver Nuggets, W2=Los Angeles Lakers:  EV=$28,120.12
E1=Chicago Bulls, E2=Boston Celtics, W1=Denver Nuggets, W2=Phoenix Suns:  EV=$24,081.24
E1=Chic